In [1]:
#!/bin/bash
!curl -L -o blood-cell-images-for-cancer-detection.zip\
  https://www.kaggle.com/api/v1/datasets/download/sumithsingh/blood-cell-images-for-cancer-detection

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  121M  100  121M    0     0  15.5M      0  0:00:07  0:00:07 --:--:-- 20.9M


In [2]:
!unzip blood-cell-images-for-cancer-detection.zip -d "blood-cell-images-for-cancer-detection"

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: blood-cell-images-for-cancer-detection/basophil/BA_100102.jpg  
  inflating: blood-cell-images-for-cancer-detection/basophil/BA_101381.jpg  
  inflating: blood-cell-images-for-cancer-detection/basophil/BA_101611.jpg  
  inflating: blood-cell-images-for-cancer-detection/basophil/BA_102039.jpg  
  inflating: blood-cell-images-for-cancer-detection/basophil/BA_10622.jpg  
  inflating: blood-cell-images-for-cancer-detection/basophil/BA_107595.jpg  
  inflating: blood-cell-images-for-cancer-detection/basophil/BA_111710.jpg  
  inflating: blood-cell-images-for-cancer-detection/basophil/BA_112418.jpg  
  inflating: blood-cell-images-for-cancer-detection/basophil/BA_114542.jpg  
  inflating: blood-cell-images-for-cancer-detection/basophil/BA_114899.jpg  
  inflating: blood-cell-images-for-cancer-detection/basophil/BA_115323.jpg  
  inflating: blood-cell-images-for-cancer-detection/basophil/BA_116477.jpg  
  inflati

In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split

source_dir = 'blood-cell-images-for-cancer-detection'
target_dir = 'blood-cell-images-for-cancer-detection-prepared'

def split_cell_dataset(source_dir, target_dir, train_ratio=0.6, val_ratio=0.2, test_ratio=0.2, random_state=42):
    """
    Split image dataset into train, validation and test sets with class preservation.

    Args:
        source_dir: Directory with class subdirectories
        target_dir: Directory where split datasets will be created
        train_ratio: Proportion for training set
        val_ratio: Proportion for validation set
        test_ratio: Proportion for test set
        random_state: Random seed for reproducibility
    """

    # Create target directory structure
    os.makedirs(target_dir, exist_ok=True)

    # Get list of class directories
    class_names = [d for d in os.listdir(source_dir)
                   if os.path.isdir(os.path.join(source_dir, d))]

    # Create split directories for each class
    for split in ['train', 'val', 'test']:
        for class_name in class_names:
            os.makedirs(os.path.join(target_dir, split, class_name), exist_ok=True)

    # Process each class
    for class_name in class_names:
        class_path = os.path.join(source_dir, class_name)
        images = os.listdir(class_path)

        # First split: train+val vs test
        train_val, test = train_test_split(
            images,
            test_size=test_ratio,
            random_state=random_state,
            shuffle=True
        )

        # Second split: train vs val
        val_relative_ratio = val_ratio / (train_ratio + val_ratio)
        train, val = train_test_split(
            train_val,
            test_size=val_relative_ratio,
            random_state=random_state,
            shuffle=True
        )

        # Copy files to split directories
        for img in train:
            src = os.path.join(class_path, img)
            dst = os.path.join(target_dir, 'train', class_name, img)
            shutil.copy(src, dst)

        for img in val:
            src = os.path.join(class_path, img)
            dst = os.path.join(target_dir, 'val', class_name, img)
            shutil.copy(src, dst)

        for img in test:
            src = os.path.join(class_path, img)
            dst = os.path.join(target_dir, 'test', class_name, img)
            shutil.copy(src, dst)


split_cell_dataset(source_dir, target_dir, train_ratio=0.6, val_ratio=0.2, test_ratio=0.2, random_state=42)

In [4]:
from PIL import Image
import numpy as np

# Load an image
img = Image.open('blood-cell-images-for-cancer-detection-prepared/train/basophil/BA_250000.jpg')

# Resize to target size
img = img.resize((224, 224))

# Convert to numpy array
x = np.array(img)
print(x.shape)  # (224, 224, 3)

(224, 224, 3)


In [5]:
import torch
import torchvision.models as models
from torchvision import transforms

In [6]:
# Load pre-trained model
model = models.mobilenet_v2(weights='IMAGENET1K_V1')
model.eval();

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 125MB/s]


In [7]:
# Preprocessing for MobileNetV2
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [8]:
img_t = preprocess(img)
batch_t = torch.unsqueeze(img_t, 0)

# Make prediction
with torch.no_grad():
    output = model(batch_t)

# Get top predictions
_, indices = torch.sort(output, descending=True)

In [9]:
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt -O imagenet_classes.txt

# Load ImageNet class names
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

# Get top 5 predictions
top5_indices = indices[0, :5].tolist()
top5_classes = [categories[i] for i in top5_indices]

print("Top 5 predictions:")
for i, class_name in enumerate(top5_classes):
    print(f"{i+1}: {class_name}")

--2025-12-30 19:34:24--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2025-12-30 19:34:24 (68.6 MB/s) - ‘imagenet_classes.txt’ saved [10472/10472]

Top 5 predictions:
1: golf ball
2: cup
3: face powder
4: envelope
5: shower curtain


In [10]:
import os
from torch.utils.data import Dataset

class BloodDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(data_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
            for img_name in os.listdir(label_dir):
                self.image_paths.append(os.path.join(label_dir, img_name))
                self.labels.append(self.class_to_idx[label_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [11]:
input_size = 224

# ImageNet normalization values
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Simple transforms - just resize and normalize
# train_transforms = transforms.Compose([
#     transforms.Resize((input_size, input_size)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=mean, std=std)
# ])

# Training transforms WITH augmentation
train_transforms = transforms.Compose([
    transforms.RandomRotation(10),           # Rotate up to 10 degrees
    transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),  # Zoom
    transforms.RandomHorizontalFlip(),       # Horizontal flip
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

# Validation transforms - NO augmentation, same as before
val_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [12]:
from torch.utils.data import DataLoader

train_dataset = BloodDataset(
    data_dir='./blood-cell-images-for-cancer-detection-prepared/train/',
    transform=train_transforms
)

val_dataset = BloodDataset(
    data_dir='./blood-cell-images-for-cancer-detection-prepared/val/',
    transform=val_transforms
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [13]:
import torch.optim as optim
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [14]:
import torch.nn as nn

class BloodClassifierMobileNet(nn.Module):
    def __init__(self, size_inner=100, droprate=0.2, num_classes=5):
        super(BloodClassifierMobileNet, self).__init__()

        self.base_model = models.mobilenet_v2(weights='IMAGENET1K_V1')

        for param in self.base_model.parameters():
            param.requires_grad = False

        self.base_model.classifier = nn.Identity()

        self.global_avg_pooling = nn.AdaptiveAvgPool2d((1, 1))
        self.inner = nn.Linear(1280, size_inner)  # New inner layer
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(droprate)  # Add dropout
        self.output_layer = nn.Linear(size_inner, num_classes)

    def forward(self, x):
        x = self.base_model.features(x)
        x = self.global_avg_pooling(x)
        x = torch.flatten(x, 1)
        x = self.inner(x)
        x = self.relu(x)
        x = self.dropout(x)  # Apply dropout
        x = self.output_layer(x)
        return x

def make_model(
        learning_rate=0.001, # best value, defined below
        size_inner=100, # best value, defined below
        droprate=0.5, # best value, defined below
):
    model = BloodClassifierMobileNet(
        num_classes=5,
        size_inner=size_inner,
        droprate=droprate,
    )
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    return model, optimizer

In [15]:
def train_and_evaluate(model, optimizer, train_loader, val_loader, criterion, num_epochs, device):
    best_val_accuracy = 0.0  # Initialize variable to track the best validation accuracy


    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_acc = correct / total

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total

        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')


        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            checkpoint_path = f'mobilenet_result_{epoch+1:02d}_{val_acc:.3f}.pth'
            torch.save(model.state_dict(), checkpoint_path)
            print(f'Checkpoint saved: {checkpoint_path}')

In [16]:
# num_epochs = 10
# learning_rates = [0.0001, 0.001, 0.01, 0.1]

# for lr in learning_rates:
#     print(f'\n=== Learning Rate: {lr} ===')
#     model, optimizer = make_model(learning_rate=lr)
#     train_and_evaluate(model, optimizer, train_loader, val_loader, criterion, num_epochs, device)

# learning_rate=0.001 is the best and most stable value

# num_epochs = 10
# sizes_inner = [1000, 500, 100]

# for size_inner in sizes_inner:
#     print(f'\n=== Size inner: {size_inner} ===')
#     model, optimizer = make_model(
#         learning_rate=0.001,
#         size_inner=size_inner
#     )
#     train_and_evaluate(model, optimizer, train_loader, val_loader, criterion, num_epochs, device)

# size_inner=100 is the best and most stable value

# num_epochs = 10
# droprates = [0.1, 0.2, 0.5, 0.7]

# for droprate in droprates:
#     print(f'\n=== Droprate: {droprate} ===')
#     model, optimizer = make_model(
#         learning_rate=0.001,
#         size_inner=100,
#         droprate=droprate,
#     )
#     train_and_evaluate(model, optimizer, train_loader, val_loader, criterion, num_epochs, device)

# droprate=0.5 is the best and most stable value



num_epochs = 50
learning_rate = 0.001
size_inner = 100
droprate = 0.5

model, optimizer = make_model(
        learning_rate=learning_rate,
        size_inner=size_inner,
        droprate=droprate,
    )
train_and_evaluate(model, optimizer, train_loader, val_loader, criterion, num_epochs, device)

Epoch 1/50
  Train Loss: 0.5194, Train Acc: 0.8300
  Val Loss: 0.1361, Val Acc: 0.9640
Checkpoint saved: mobilenet_result_01_0.964.pth
Epoch 2/50
  Train Loss: 0.1775, Train Acc: 0.9477
  Val Loss: 0.1056, Val Acc: 0.9640
Epoch 3/50
  Train Loss: 0.1449, Train Acc: 0.9580
  Val Loss: 0.1519, Val Acc: 0.9380
Epoch 4/50
  Train Loss: 0.1131, Train Acc: 0.9660
  Val Loss: 0.0784, Val Acc: 0.9720
Checkpoint saved: mobilenet_result_04_0.972.pth
Epoch 5/50
  Train Loss: 0.1114, Train Acc: 0.9670
  Val Loss: 0.0802, Val Acc: 0.9740
Checkpoint saved: mobilenet_result_05_0.974.pth
Epoch 6/50
  Train Loss: 0.1262, Train Acc: 0.9580
  Val Loss: 0.0832, Val Acc: 0.9700
Epoch 7/50
  Train Loss: 0.1015, Train Acc: 0.9653
  Val Loss: 0.0634, Val Acc: 0.9740
Epoch 8/50
  Train Loss: 0.0891, Train Acc: 0.9720
  Val Loss: 0.0682, Val Acc: 0.9720
Epoch 9/50
  Train Loss: 0.0847, Train Acc: 0.9693
  Val Loss: 0.0571, Val Acc: 0.9810
Checkpoint saved: mobilenet_result_09_0.981.pth


KeyboardInterrupt: 

In [17]:
path = '/content/mobilenet_result_09_0.981.pth' # Validation accuracy for the trained model is more than 98%

In [18]:
model = BloodClassifierMobileNet(size_inner=100, droprate=0.5, num_classes=5)
model.load_state_dict(torch.load(path))
model.to(device)
model.eval();

In [19]:
x = val_transforms(img)
batch_t = torch.unsqueeze(x, 0).to(device)

# Make prediction
with torch.no_grad():
    output = model(batch_t)

In [20]:
train_dataset.class_to_idx

{'basophil': 0,
 'erythroblast': 1,
 'monocyte': 2,
 'myeloblast': 3,
 'seg_neutrophil': 4}

In [21]:
classes = [
    "basophil",
    "erythroblast",
    "monocyte",
    "myeloblast",
    "seg_neutrophil"
]

In [23]:
dict(zip(classes, output[0].to('cpu')))

{'basophil': tensor(6.9623),
 'erythroblast': tensor(-4.6289),
 'monocyte': tensor(-0.3891),
 'myeloblast': tensor(-7.3883),
 'seg_neutrophil': tensor(-7.3689)}

In [ ]:
# 'basophil' class has the highest value - this is correct prediction

In [24]:
# Checking of trained model with the image from test part of dataset

from PIL import Image
import numpy as np

# Load an image from test part of the prepared dataset
img_test = Image.open('blood-cell-images-for-cancer-detection-prepared/test/myeloblast/MYO_0911.jpg')

# Resize to target size
img_test = img_test.resize((224, 224))

x = val_transforms(img_test)
batch_t = torch.unsqueeze(x, 0).to(device)

# Make prediction
with torch.no_grad():
    output = model(batch_t)

dict(zip(classes, output[0].to('cpu')))

{'basophil': tensor(-6.9226),
 'erythroblast': tensor(-5.7898),
 'monocyte': tensor(-3.0539),
 'myeloblast': tensor(11.8165),
 'seg_neutrophil': tensor(1.5645)}

In [25]:
# 'myeloblast' class has the highest value - this is correct for the test image, prediction of the high risk of cancer by AML indicators

In [26]:
!pip install onnx onnxscript

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.4/693.4 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 8.6 MB/s eta 0:00:00


In [28]:
# Create dummy input
dummy_input = torch.randn(1, 3, 224, 224).to(device)

# Export to ONNX
onnx_path = "blood_classifier_mobilenet_result.onnx"

torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    verbose=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    },
    external_data=False # used to compose .onnx and .onnx.data files into single file
)

print(f"Model exported to {onnx_path}")


/tmp/ipython-input-1634812886.py:7: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(


[torch.onnx] Obtain model graph for `BloodClassifierMobileNet([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `BloodClassifierMobileNet([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 105 of general pattern rewrite rules.
Model exported to blood_classifier_mobilenet_result.onnx
